### Data Collection and Pre-processing

In [1]:
# Installs without warnings in conda env: dsi
!pip install visual_genome

  Using cached visual_genome-1.1.1-py2.py3-none-any.whl (9.6 kB)
  Using cached progressbar2-3.53.1-py2.py3-none-any.whl (25 kB)
  Using cached python_utils-2.4.0-py2.py3-none-any.whl (12 kB)


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import json

# Import the Python wrapper for the VG API
from visual_genome import api

In [3]:
# Retrieve an array of integers representing all VG image ID's
image_ids = api.get_all_image_ids()

with open('../data/image_ids.txt', 'w') as imagehandles:
    imagehandles.writelines("%s\n" % ids for ids in image_ids)

The processing method provided by the Visual Genome API, using relation mappings, is too slow to be effective in obtaining the object descriptions.

In [8]:
# Read objects.json file from Visual Genome
with open('../../objects.json', 'r') as objects_json_file:
    # List with len(json_objects) = 108_077 dictionaries representing images
    json_objects = json.load(objects_json_file)

In [9]:
# loop thru each item in list
dog_pic_ids = []
for image_index in range(len(json_objects)):
    # Number of objects
    # len(json_objects[image_index]) = 3
    # json_objects[image_index].keys() = dict_keys(['image_id', 'objects', 'image_url'])
    num_objects = len(json_objects[image_index]['objects'])
    # loop through objects_index searching for dog objects
    objects_index = 0
    # Objects_index lives in range(num_objects)
    while objects_index < num_objects:
        # Save image_id's of images with dogs
        if json_objects[image_index]['objects'][objects_index][ 'names' ] == [  'dog'  ]:
      # if json_objects[image_index]['objects'][objects_index]['synsets'] == ['dog.n.01']:
    #####NOTICE: INSERT HOT DOG FILTER HERE LOL!
            # Save image_id's of images with dogs
            image_id = json_objects[image_index]['image_id']
            dog_pic_ids.append(image_id)
            # Escape 'while-loop' as soon as 'dog'-object found
            objects_index = num_objects
        else:
            # Move on to next object in image, continue searching
            objects_index += 1
# Check how many dog pics were discovered in the dataset
print(len(dog_pic_ids))

3235


In [10]:
if 308 in dog_pic_ids[:10]:
    print('True')

True


In [3]:
root = 'VG_100K_2/2377381'
print(root[10:])

2377381


In [11]:
ids;

In [12]:
from zipfile import ZipFile
import numpy as np
import cv2
# Import Python's standard utility module 'os' for interacting with Operating System
import os
# NOTICE CODE FROM
# https://thispointer.com/python-how-to-get-the-list-of-all-files-in-a-zip-archive/


# ~10GB ZIP Archive, 1/3 of total data set
with ZipFile('../../visual_genome_part2.zip', "r") as z:
    # One file in zip archive
    VG_100K_2 = z.namelist()

    # Iterate over image file names, 'VG_100K_2/image_id.jpg'
    for ith_image in VG_100K_2: 
        # Get extension of file, '.jpg'
        ext = os.path.splitext(ith_image)[-1]
        # Get root of file, root = VG_100K_2/image_id.jpg
        root = os.path.splitext(ith_image)[0]
        
        # Skip over Archive Directory
        if (ext == ".jpg"):
            # Skip 'VG_100K_2' root[10:] in dog_pic_ids
            if int(root[10:]) in dog_pic_ids:
                # Read image binary data of 'VG_100K_2' from zip('visual_genome_part2.zip')
                in_bytes = z.read(ith_image) # VG_100K_2/
                # Decode bytes to image
                img = cv2.imdecode(np.frombuffer(in_bytes, np.uint8), cv2.IMREAD_COLOR)
    
                # Show image for testing
                cv2.imshow('img', img)
                # Display for 1sec = 1_000ms
                cv2.waitKey(1000)

cv2.destroyAllWindows()

In [59]:
import os
import zipfile

# importing OpenCV(cv2) module 
import cv2 

with zipfile.ZipFile('../../visual_genome_part2.zip', 'r') as z:
    for filename in z.namelist():
        if not os.path.isdir(filename):
            print(filename)
            # read the file
            with z.open(filename) as f:
                print(filename)
                #imgdata = f.read('617.jpg')
                # Save image in set directory 
                # Load RGB image 
                img = cv2.imread('617.jpg')
               
                # Output img with window name as 'image' 
                cv2.imshow('image', img)  

                # Maintain output window utill 
                # user presses a key 
                cv2.waitKey(0)         

                # Destroying present windows on screen 
                cv2.destroyAllWindows()

VG_100K_2/
VG_100K_2/


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wwma2wne\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [51]:
import zipfile
import numpy as np
import os
# importing OpenCV(cv2) module 
import cv2 

with zipfile.ZipFile('../../visual_genome_part2.zip', 'r') as z2:
    for filename in z2.namelist():
        if not os.path.isdir(filename):
            # read the file
            with z2.open(filename) as f:
                #imgdata = f.read('617.jpg')
                # Save image in set directory 
                # Read RGB image 
                img = cv2.imread('617.jpg')
                #img = cv2.imdecode(np.frombuffer(imgdata, np.uint8), 1) 
                # Output img with window name as 'image' 
                cv2.imshow('image', img)  

                # Maintain output window utill 
                # user presses a key 
                #cv2.waitKey(0)         

                # Destroying present windows on screen 
                #cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wwma2wne\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [49]:
with zipfile.ZipFile('../../visual_genome_part2.zip', 'r') as zfile:
    data = f.read('1.jpg')

img = cv2.imdecode(np.frombuffer(data, np.uint8), 1)    

TypeError: '<' not supported between instances of 'str' and 'int'

In [48]:
import os
import zipfile

# importing matplotlib modules 
import matplotlib.image as mpimg

with zipfile.ZipFile('../../visual_genome_part2.zip', 'r') as z2:
    for filename in z2.namelist():
        if not os.path.isdir(filename):
            # read the file
            with z2.open(filename) as f:
                #imgdata = f.read('617.jpg')
                # Save image in set directory 
                # Read RGB image 
                img = mpimg.imread('617.jpg')
                # Output img with window name as 'image' 
                plt.imshow('image', img)

FileNotFoundError: [Errno 2] No such file or directory: '617.jpg'

In [42]:
type(img)

NoneType

In [36]:
# Output img with window name as 'image' 
cv2.imshow('image', img)  
  
# Maintain output window utill 
# user presses a key 
cv2.waitKey(0)         
  
# Destroying present windows on screen 
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wwma2wne\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [15]:
json_objects[617]
dog_pic_ids[0]

int

In [ ]:
# Hot Dog filter
#for dog in dog_pic_ids:
    # Consider what 